In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt


font = {'font.family' : 'normal',
        'font.weight' : 'normal',
        'font.size'   : 20}
plt.rcParams.update(font)

# Logistic Map

Until now we have considered dynamical systems in which time is a *continuous* quantity. These systems are described by differential equations.
Another class of dynamical systems assumes time to be a *discrete* quantity. Such systems are known as difference equations, recursion relations, iterated maps or simply **maps**.

An interesting example of such systems is the so-called **logistic map** which is a *discrete-time* analog of the logistic equation for population growth:

$$x_{n+1} = r\, x_n\,(1-x_n)$$

with $x_n \geq 0$ a dimensionless measure of the population in the $n$-th generation, and $r \geq 0$ the intrinsic growth rate.



## Model

In [ ]:
def log_map(x_previous, r):
    """Computes population of generation n+1 from generation n"""
    x_next = r * x_previous * (1.0-x_previous)
    return x_next


In [ ]:
r = 0.1
x_p = np.arange(0,1,0.01)
x_n = log_map(x_p, r=r)

fig, ax = plt.subplots(figsize=(15, 7))
ax.plot(x_p, x_n)
ax.grid()
ax.set_xlabel('$x_n$')
ax.set_ylabel('$x_{n+1}$')
ax.set_title("logistic map for r=%.2f"%(r))

ax.axhline(y=r/4., color='red', linestyle=':', label='max @ $r/4=$%.2f'%(r/4))
ax.legend(loc='upper right')


## Evolution 

In [ ]:
def compute_log_map(r, x0=0.1, n_max=100):
    """Computes logistic map over n_max generations"""
    x = np.zeros(n_max)
    x[0] = x0
    for n in range(1,n_max):
        x[n] = log_map(x[n-1],r)
    return x

What behavior do you observe in the following scenarios:

- $r<1$
- $1<r<3$
- $r > 3$
    - $r=3.3$
    - $r=3.5$

In [ ]:
y = compute_log_map(r=2.9, x0=0.3, n_max=50)
x = np.arange(0,len(y))
plt.plot(x, y,'-x')

# n-period
#n=32
#plt.plot(x[::n], y[::n], '.r')


- For $r<1$ population goes extinct, $x_n\rightarrow 0$ as $n \rightarrow \infty$. 
- For $1<r<3$ population grows and eventually reaches a nonzero steady state.
- For $r>3$ population builds up and then oscillates about the former steady state, with different *periods* dependening on the value of $r$, e.g. *period-2* cycle for $r=3.3$, *period-4* cycle for $r=3.5$, and further *period-doublings* with higher values of $r$ up to a limiting value $r_\infty\approx 3.569946$.

For many values of $r>r_\infty$, the long-term behavior is aperiodic.
    

## Cobweb Diagram

*[Cobweb plots](https://en.wikipedia.org/wiki/Cobweb_plot)* are often used it visualize the behavior of iterated maps:
$$x_{n+1} = f(x_n)$$

They are constructed using two curves:
- the diagonal $y=x_{n+1} = x_n$
- the function curve $y=f(x_n)$

To plot the behavior of a starting value $x_0$, the following steps are applied:

1. find point with coordinates $(x_0, f(x_0))$
2. plot horizontally from this point to the diagonal line to find the intersection point $(f(x_0), f(x_0))$
3. plot vertically from the point on the diagonal to the function curve: $(f(x_0), f(f(x_0)))$
4. repeat from 2. as needed


In [ ]:

def plot_cobweb(f=log_map, r=0.1, x0=0.5, nmax=40):
    # adapted from https://scipython.com/blog/cobweb-plots/
    """Cobweb plot

    Plot y = f(x; r) and y = x for 0 <= x <= 1.
    Illustrate the behavior of iterating x = f(x) starting at x = x0. 
    r is a parameter of the function f.
    """
    x = np.linspace(0, 1, 500)
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)

    # Plot y = f(x) and y = x
    ax.plot(x, f(x, r), color='k', lw=2)
    ax.plot(x, x, color='k', linestyle=':',lw=2)

    # Iterate x = f(x) for nmax steps, starting at (x0, 0).
    px, py = np.empty((2,nmax+1,2))
    px[0], py[0] = x0, 0
    for n in range(1, nmax, 2):
        px[n] = px[n-1]
        py[n] = f(px[n-1], r)
        px[n+1] = py[n]
        py[n+1] = py[n]

    # Plot the path traced out by the iteration.
    ax.plot(px, py, color='b', alpha=0.5,lw=1)
    ax.plot(px, py, linestyle='', marker='.', color='r', markersize=1)
    
    # fix points
    xf_0 = 0
    xf_1 = 1.-1./r
    ax.plot(xf_0, xf_0, marker='o', color='g')
    ax.plot(xf_1, xf_1, marker='o', color='g')

    # Annotate and tidy the plot.
    ax.minorticks_on()
    ax.grid(which='minor', alpha=0.5)
    ax.grid(which='major', alpha=0.5)
    ax.set_aspect('equal')
    ax.set_xlabel('$x_n$')
    ax.set_ylabel('$x_{n+1}$')
    ax.set_title('$x_0 = {:.1}, r = {:.2}$'.format(float(x0), float(r)))
    ax.set_xlim(-0.05, 1.05)
    ax.set_ylim(-0.05, 1.05)
    plt.show()
    

In [ ]:
#plot_cobweb(log_map, r=1.5, x0=0.01, nmax=1000)

In [ ]:
# interactive ipython widgets
import ipywidgets as widgets
from IPython.display import display


#-- initial values
w_x0 = widgets.FloatSlider(value=0.1, min=0.0, max=1.0, step=0.01, continuous_update=False, 
                                   description=r'\(x_0\)', orientation='horizontal', readout=True)
w_r  = widgets.FloatSlider(value=1.5, min=0.0, max=5.0, step=0.01, continuous_update=False, 
                           description=r'\(r\)', orientation='horizontal', readout=True)
w_n  = widgets.IntSlider(value=500, min=0, max=1000, step=1, continuous_update=False, 
                         description=r'\(n\)', orientation='horizontal', readout=True)
ui = widgets.HBox([w_x0,w_r,w_n])

out = widgets.interactive_output(plot_cobweb, { 
                                                'r' : w_r,
                                                'x0': w_x0,
                                                'nmax' : w_n    })

# # display widget
display(ui, out)

[Link](https://commons.wikimedia.org/wiki/File:LogisticCobwebChaos.gif) to animated cobweb diagram.

## Orbit Diagram

The longterm behavior of the iterative map for *all* values of parameter $r$ can be represented in an *orbit diagram* that plots the system's attractor as a function of $r$.

To generate this plot, we 
- solve the logistic map for some random initial condition $x_0$ and iterate over a sufficiently large number of generations for the system to settle.
- repeat this for all values $r$ in the interval of interest,
- plot all solutions as function of $r$, excluding 'early' generations.

In [ ]:
# generate r_steps number of orbits, each with n_generations generations
r_steps = 500
n_generations = 1000
x0 = 0.1


rs = np.linspace(0, 4, r_steps)
y  = np.zeros((r_steps, n_generations))*np.NaN
for i, r in enumerate(rs):
    y[i,:] = compute_log_map(r, x0=0.2, n_max=n_generations)
    
    
# plot orbits in function of r
fig, ax = plt.subplots(figsize=(15, 15))
ax.plot(rs, y[:,50:], '.k', markersize=0.1)
#ax.plot(rs, y[:,:], '.k', markersize=0.1)
ax.set_xlabel('$r$')
ax.set_ylabel('$x$')
ax.grid(which='minor', alpha=0.5)
ax.grid(which='major', alpha=0.5)
ax.minorticks_on()
plt.show()

## Analysis

Logistic map:
$$x_{n+1} = F(x_n) = r\, x_n\,(1-x_n)$$

### Fix Points

Fix points satisfy 
$$x^* = F(x^*) = r\, x^*\,(1-x^*)$$

Hence:
1. $x^* =0$
2. $x^* = r\, x^*\,(1-x^*)\, \rightarrow \, x^*=1-\frac{1}{r}$;
   $x^*$ only in range of allowable $x$ if $r\geq 1$ 


### Bifurcation

Fixpoint $x^*$ is 
- *linearly stable* if the eigenvalue $\left| \lambda \right|=\left|F'(x^*)\right| <1$ 
- *unstable* if the eigenvalue $\left| \lambda \right|=\left|F'(x^*)\right| >1$. 

*Bifurcation* at $x^*$ if $\left| \lambda \right|=\left|F'(x^*)\right| =1$.

$F'(x) = r - 2\, r\, x = r\, (1-2x)$:
- $F'(x^*=0) = r$
    - stable if $ r <1 $
    - unstable if $r>1$
- $F'(x^*=1-\frac{1}{r}) = r\,(1-2\, (1-\frac{1}{r})) = 2- r$
    - stable if $ -1 < (2-r) <1 $, i.e. $1 < r < 3$
    - unstable if $r>3$
